# Deepfake Detection Platform - Image Models Test

This notebook tests all image detection models and the Visual Sentinel Singularity Mode.

In [11]:
# Setup imports
import os
import sys
import logging
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch

# Add the project root to the path
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), '..'))
sys.path.insert(0, project_root)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [12]:
# Check CUDA availability
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

CUDA available: False


## 1. Load Individual Image Detector Models

We'll load all the image detector models: ViT, BEIT, DeiT, and Swin.

In [ ]:
# Import image detector models
from detectors.image_detector.vit_detector import ViTImageDetector
from detectors.image_detector.beit_detector import BEITImageDetector
from detectors.image_detector.deit_detector import DeiTImageDetector
from detectors.image_detector.swin_detector import SwinImageDetector

# Define the device to use
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Initialize the image detectors
try:
    print("Loading ViT model...")
    vit_detector = ViTImageDetector(model_name="google/vit-base-patch16-224", 
                                  confidence_threshold=0.5,
                                  device=device)
    
    print("Loading BEIT model...")
    beit_detector = BEITImageDetector(model_name="microsoft/beit-base-patch16-224-pt22k-ft22k", 
                                   confidence_threshold=0.5,
                                   device=device)
    
    print("Loading DeiT model...")
    deit_detector = DeiTImageDetector(model_name="facebook/deit-base-distilled-patch16-224", 
                                   confidence_threshold=0.5,
                                   device=device)
    
    print("Loading Swin model...")
    swin_detector = SwinImageDetector(model_name="microsoft/swin-base-patch4-window7-224-in22k", 
                                   confidence_threshold=0.5,
                                   device=device)
    
    print("All models loaded successfully!")
    
except Exception as e:
    print(f"Error loading models: {str(e)}")

ImportError: cannot import name 'AutoModelForImageClassification' from 'transformers' (c:\Users\chakr\anaconda3\Lib\site-packages\transformers\__init__.py)

: 

## 2. Test Individual Models on Sample Images

We'll test each model on sample real and fake images.

In [ ]:
# Define test images
real_image_path = os.path.join(project_root, 'tests', 'test_data', 'Real_Image', 'real_993.jpg')
fake_image_path = os.path.join(project_root, 'tests', 'test_data', 'Fake_Image', 'fake_999.jpg')

# Check if images exist
if not os.path.exists(real_image_path):
    raise FileNotFoundError(f"Real image not found: {real_image_path}")
if not os.path.exists(fake_image_path):
    raise FileNotFoundError(f"Fake image not found: {fake_image_path}")

# Display the test images
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
axs[0].imshow(Image.open(real_image_path))
axs[0].set_title("Real Image")
axs[0].axis('off')
axs[1].imshow(Image.open(fake_image_path))
axs[1].set_title("Fake Image")
axs[1].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Helper function to test models on images
def test_model(detector, image_path, image_type):
    print(f"Testing {detector.__class__.__name__} on {image_type} image...")
    try:
        result = detector.detect(image_path)
        is_deepfake = result['is_deepfake']
        confidence = result['confidence']
        correct = (is_deepfake and image_type == 'fake') or (not is_deepfake and image_type == 'real')
        
        print(f"  Prediction: {'Deepfake' if is_deepfake else 'Real'}")
        print(f"  Confidence: {confidence:.4f}")
        print(f"  Correct: {'✓' if correct else '✗'}")
        
        if 'heatmap' in result:
            plt.figure(figsize=(8, 4))
            plt.subplot(1, 2, 1)
            plt.imshow(Image.open(image_path))
            plt.title(f"{image_type.capitalize()} Image")
            plt.axis('off')
            
            plt.subplot(1, 2, 2)
            plt.imshow(result['heatmap'], cmap='hot')
            plt.title(f"Heatmap (Confidence: {confidence:.4f})")
            plt.axis('off')
            plt.colorbar()
            plt.tight_layout()
            plt.show()
        
        return result, correct
    except Exception as e:
        print(f"  Error: {str(e)}")
        return None, False

In [ ]:
# Test ViT model
vit_real_result, vit_real_correct = test_model(vit_detector, real_image_path, 'real')
vit_fake_result, vit_fake_correct = test_model(vit_detector, fake_image_path, 'fake')

In [ ]:
# Test BEIT model
beit_real_result, beit_real_correct = test_model(beit_detector, real_image_path, 'real')
beit_fake_result, beit_fake_correct = test_model(beit_detector, fake_image_path, 'fake')

In [ ]:
# Test DeiT model
deit_real_result, deit_real_correct = test_model(deit_detector, real_image_path, 'real')
deit_fake_result, deit_fake_correct = test_model(deit_detector, fake_image_path, 'fake')

In [ ]:
# Test Swin model
swin_real_result, swin_real_correct = test_model(swin_detector, real_image_path, 'real')
swin_fake_result, swin_fake_correct = test_model(swin_detector, fake_image_path, 'fake')

## 3. Test Image Ensemble Detector

Now we'll test the ensemble detector that combines all image models.

In [ ]:
# Import the image ensemble detector
from detectors.image_detector.ensemble import ImageEnsembleDetector

# Create the ensemble detector with all individual detectors
detectors = [vit_detector, beit_detector, deit_detector, swin_detector]
image_ensemble = ImageEnsembleDetector(
    detectors=detectors,
    weights=None,  # Use equal weights initially
    threshold=0.5,
    enable_singularity=False  # First test without Singularity Mode
)

print(f"Created Image Ensemble Detector with {len(detectors)} models")

In [ ]:
# Test ensemble on real image
print("Testing Image Ensemble on real image...")
ensemble_real_result = image_ensemble.predict(real_image_path)
is_deepfake = ensemble_real_result['is_deepfake']
confidence = ensemble_real_result['confidence']
print(f"  Prediction: {'Deepfake' if is_deepfake else 'Real'}")
print(f"  Confidence: {confidence:.4f}")
print(f"  Correct: {'✓' if not is_deepfake else '✗'}")

# Show individual model contributions if available
if 'individual_results' in ensemble_real_result:
    print("\nIndividual model contributions:")
    for result in ensemble_real_result['individual_results']:
        model_name = result['model']
        confidence = result['confidence']
        weight = result['weight']
        print(f"  {model_name}: Confidence = {confidence:.4f}, Weight = {weight:.2f}")

In [ ]:
# Test ensemble on fake image
print("Testing Image Ensemble on fake image...")
ensemble_fake_result = image_ensemble.predict(fake_image_path)
is_deepfake = ensemble_fake_result['is_deepfake']
confidence = ensemble_fake_result['confidence']
print(f"  Prediction: {'Deepfake' if is_deepfake else 'Real'}")
print(f"  Confidence: {confidence:.4f}")
print(f"  Correct: {'✓' if is_deepfake else '✗'}")

# Show individual model contributions if available
if 'individual_results' in ensemble_fake_result:
    print("\nIndividual model contributions:")
    for result in ensemble_fake_result['individual_results']:
        model_name = result['model']
        confidence = result['confidence']
        weight = result['weight']
        print(f"  {model_name}: Confidence = {confidence:.4f}, Weight = {weight:.2f}")

## 4. Test Visual Sentinel Singularity Mode

Now we'll test the Visual Sentinel Singularity Mode for enhanced image detection.

In [ ]:
# Enable Singularity Mode in the ensemble detector
image_ensemble_with_singularity = ImageEnsembleDetector(
    detectors=detectors,
    weights=None,
    threshold=0.5,
    enable_singularity=True  # Enable Singularity Mode
)

print("Created Image Ensemble Detector with Visual Sentinel Singularity Mode enabled")

In [ ]:
# Test Singularity Mode on real image
print("Testing Visual Sentinel on real image...")
try:
    singularity_real_result = image_ensemble_with_singularity.predict(real_image_path)
    is_deepfake = singularity_real_result['is_deepfake']
    confidence = singularity_real_result['confidence']
    print(f"  Prediction: {'Deepfake' if is_deepfake else 'Real'}")
    print(f"  Confidence: {confidence:.4f}")
    print(f"  Correct: {'✓' if not is_deepfake else '✗'}")
    
    # Show Singularity Mode information if available
    if 'singularity_mode' in singularity_real_result:
        sm_info = singularity_real_result['singularity_mode']
        print("\nSingularity Mode information:")
        for key, value in sm_info.items():
            if key != 'adaptive_weights':  # Weights would be too verbose
                print(f"  {key}: {value}")
        
        if 'adaptive_weights' in sm_info:
            print("  Adaptive weights:")
            for model, weight in sm_info['adaptive_weights'].items():
                print(f"    {model}: {weight:.4f}")
                
    # Display enhanced heatmap if available
    if 'heatmap' in singularity_real_result:
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(Image.open(real_image_path))
        plt.title("Real Image")
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(singularity_real_result['heatmap'], cmap='hot')
        plt.title(f"Enhanced Heatmap (Confidence: {confidence:.4f})")
        plt.axis('off')
        plt.colorbar()
        plt.tight_layout()
        plt.show()
        
except Exception as e:
    print(f"Error testing Visual Sentinel: {str(e)}")

In [ ]:
# Test Singularity Mode on fake image
print("Testing Visual Sentinel on fake image...")
try:
    singularity_fake_result = image_ensemble_with_singularity.predict(fake_image_path)
    is_deepfake = singularity_fake_result['is_deepfake']
    confidence = singularity_fake_result['confidence']
    print(f"  Prediction: {'Deepfake' if is_deepfake else 'Real'}")
    print(f"  Confidence: {confidence:.4f}")
    print(f"  Correct: {'✓' if is_deepfake else '✗'}")
    
    # Show Singularity Mode information if available
    if 'singularity_mode' in singularity_fake_result:
        sm_info = singularity_fake_result['singularity_mode']
        print("\nSingularity Mode information:")
        for key, value in sm_info.items():
            if key != 'adaptive_weights':  # Weights would be too verbose
                print(f"  {key}: {value}")
        
        if 'adaptive_weights' in sm_info:
            print("  Adaptive weights:")
            for model, weight in sm_info['adaptive_weights'].items():
                print(f"    {model}: {weight:.4f}")
                
    # Display enhanced heatmap if available
    if 'heatmap' in singularity_fake_result:
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(Image.open(fake_image_path))
        plt.title("Fake Image")
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(singularity_fake_result['heatmap'], cmap='hot')
        plt.title(f"Enhanced Heatmap (Confidence: {confidence:.4f})")
        plt.axis('off')
        plt.colorbar()
        plt.tight_layout()
        plt.show()
        
except Exception as e:
    print(f"Error testing Visual Sentinel: {str(e)}")

## 5. Performance Comparison

Compare the performance of individual models vs. ensemble vs. Singularity Mode.

In [ ]:
# Collect all confidence scores
real_image_confidences = {
    'ViT': vit_real_result['confidence'] if vit_real_result else 0,
    'BEIT': beit_real_result['confidence'] if beit_real_result else 0,
    'DeiT': deit_real_result['confidence'] if deit_real_result else 0,
    'Swin': swin_real_result['confidence'] if swin_real_result else 0,
    'Ensemble': ensemble_real_result['confidence'],
    'Visual Sentinel': singularity_real_result['confidence']
}

fake_image_confidences = {
    'ViT': vit_fake_result['confidence'] if vit_fake_result else 0,
    'BEIT': beit_fake_result['confidence'] if beit_fake_result else 0,
    'DeiT': deit_fake_result['confidence'] if deit_fake_result else 0,
    'Swin': swin_fake_result['confidence'] if swin_fake_result else 0,
    'Ensemble': ensemble_fake_result['confidence'],
    'Visual Sentinel': singularity_fake_result['confidence']
}

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Real image confidences
models = list(real_image_confidences.keys())
confidences = list(real_image_confidences.values())
axes[0].bar(models, confidences, color=['blue', 'blue', 'blue', 'blue', 'green', 'red'])
axes[0].set_title('Real Image Confidence Scores')
axes[0].set_xlabel('Model')
axes[0].set_ylabel('Confidence (lower is better for real)')
axes[0].set_ylim(0, 1)
axes[0].axhline(y=0.5, color='r', linestyle='--')
axes[0].set_xticklabels(models, rotation=45)

# Fake image confidences
models = list(fake_image_confidences.keys())
confidences = list(fake_image_confidences.values())
axes[1].bar(models, confidences, color=['blue', 'blue', 'blue', 'blue', 'green', 'red'])
axes[1].set_title('Fake Image Confidence Scores')
axes[1].set_xlabel('Model')
axes[1].set_ylabel('Confidence (higher is better for fake)')
axes[1].set_ylim(0, 1)
axes[1].axhline(y=0.5, color='r', linestyle='--')
axes[1].set_xticklabels(models, rotation=45)

plt.tight_layout()
plt.show()

## 6. Conclusion

Summarize the test results and performance of the image detection models and Singularity Mode.

In [ ]:
# Calculate accuracy and improvement
# For real image (lower confidence is better)
real_image_improvement = max(0, np.mean([v for k, v in real_image_confidences.items() if k not in ['Ensemble', 'Visual Sentinel']]) - real_image_confidences['Visual Sentinel'])

# For fake image (higher confidence is better)
fake_image_improvement = max(0, fake_image_confidences['Visual Sentinel'] - np.mean([v for k, v in fake_image_confidences.items() if k not in ['Ensemble', 'Visual Sentinel']]))

print("Test Results Summary:")
print(f"Visual Sentinel performance improvement on real image: {real_image_improvement:.4f} (lower confidence is better)")
print(f"Visual Sentinel performance improvement on fake image: {fake_image_improvement:.4f} (higher confidence is better)")
print(f"Average improvement: {(real_image_improvement + fake_image_improvement) / 2:.4f}")